In [ ]:

from openai import OpenAI
import os
from dotenv import load_dotenv
from json_helpers import extract_json

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai = OpenAI(api_key=OPENAI_API_KEY)

In [6]:
def get_weather(city: str) -> str:
    if city == "California":
        return "Sunny"
    if city == "New York":
        return "Rainy"
    if city == "Texas":
        return "Hot"
    if city == "Florida":
        return "Humid"
    
available_actions = {
    "get_weather": get_weather,
}

In [ ]:
def generate_text(prompt:str, system_prompt: str, model:str="gpt-4.1", max_tokens:int=100) -> str:
    """
    Generate text using OpenAI's API.

    Args:
        prompt (str): The input prompt for text generation.
        system_prompt (str): The system prompt to guide the generation.
        model (str): The model to use for generation.
        max_tokens (int): The maximum number of tokens to generate.

    Returns:
        str: The generated text.
    """
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
            {"role": "system", "content": system_prompt}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

In [ ]:
def generate_text_with_conversation(messages: list, model:str="gpt-4.1", max_tokens:int=100) -> str:
    """
    Generate text using OpenAI's API with a conversation history.

    Args:
        messages (list): A list of messages in the conversation.
        model (str): The model to use for generation.
        max_tokens (int): The maximum number of tokens to generate.

    Returns:
        str: The generated text.
    """
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

In [ ]:
react_system_prompt = """
DO NOT ANSWER ANY QUESTIONS THAT ARE NOT ABOUT WEATHER OR RELATED TO WEATHER.

You are in a loop of THOUGHT, ACTION, PAUSE, ACTION_RESPONSE.
At the end of the loop you output the ANSWER.

Use THOUGHT to understand the question you have been asked.
Use ACTION to run one of the actions available to you - then return PAUSE.
ACTION_RESPONSE will be the result of running those actions.

Your available actions are:
get_weather:
e.g. get_weather: California
Returns the current weather in the specified city.

Example session:
QUESTION: Should I take an umbrella with me today in California?
THOUGHT: I need to check the weather in California to determine if an umbrella is needed.
ACTION:
{
    "function_name": "get_weather",
    "function_params": {
        "city": "California"
    }
}

PAUSE
You will be called again with this:
ACTION_RESPONSE: The weather in California is Sunny.
You then output:
ANSWER: No, you do not need to take an umbrella with you today in California.
""".strip()

In [ ]:
prompt = f"""
Should I take an umbrella when going out today in California?
"""

messages = [
    {"role": "system", "content": react_system_prompt},
    {"role": "user", "content": prompt},
]

turn_count = 1
max_turns = 5

while turn_count < max_turns:
    print(f"Loop: {turn_count}")
    print("--------------------------")
    turn_count += 1

    response = generate_text(messages=messages, model="gpt-4.1", max_tokens=1000)

    print(f"response from the model: {response}")

    json_function = extract_json(response)

    print(f"extracted json functions: {json_function}")

    if json_function:
        function_name = json_function.get("function_name")
        function_params = json_function.get("function_params", {})

        if function_name not in available_actions:
            raise ValueError(f"UNKNOWN action: {function_name}: {function_params}")

        print(f" -- running {function_name} with params: {function_params}")

        action_function = available_actions[function_name]
        result = action_function(**function_params)
        function_result_message = f"ACTION_RESPONSE: {result}"
        
        print(f" -- result: {function_result_message}")

        messages.append({"role": "user", "content": function_result_message})
        # messages = [
        #     {"role": "system", "content": react_system_prompt},
        #     {"role": "user", "content": prompt},
        #     {"role": "user", "content": function_result_message}
        # ]
    else:
        function_result_message = "No function called"
        break

    print(f"ANSWER: {function_result_message}")